In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk

from wordcloud import WordCloud
from google_play_scraper import reviews_all, Sort
from nltk.corpus import stopwords

from konlpy.tag import Okt
import jpype
jpype.startJVM(jpype.getDefaultJVMPath())
okt = Okt()

# 리뷰 데이터 수집
app_id = 'com.velox.hkmc_tm1k'  
reviews = reviews_all(
    app_id,
    lang='ko',  
    country='kr', 
    sort=Sort.NEWEST  
)

# 리뷰 데이터프레임 생성 및 분류
df = pd.DataFrame(reviews)
# df.to_csv('reviews.csv', index=False)
positive_reviews = df[df['score'] == 5].head(1000)
neutral_reviews = df[df['score'].isin([3, 4])].head(1000)
negative_reviews = df[df['score'].isin([1, 2])].head(1000)     # .isin() : 여러개의 값이 있는지 확인하는 함수

# 텍스트 전처리 함수 정의
nltk.download('stopwords')

# NLTK stopwords 한글 불용어 리스트 추가 (직접 정의)
korean_stop_words = [
    '이', '그', '저', '것', '수', '들', '및', '그것', '더', '그리고', '하지만', '그러나',
    '그래서', '또한', '및', '에서', '으로', '와', '과', '도', '는', '가', '에',
    '을', '를', '의', '은', '는', '이', '로', '하여', '했다', '이다', '있다', '없다'
]


def preprocess_text(text):
    # 특수문자 제거
    text = re.sub(r'[^\w\s]', '', text)
    # 소문자 변환
    text = text.lower()
    # 불용어 제거
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

# 한국어 전용 함수 
def preprocess_text_korean(text):
    # 특수문자 제거
    text = re.sub(r'[^\w\s]', '', text)
    # 형태소 분석 후 명사 추출
    tokens = okt.nouns(text)
    # 불용어 제거
    tokens = [word for word in tokens if word not in korean_stop_words and len(word) > 1]
    # 공백으로 단어 결합
    return ' '.join(tokens)


# 워드 클라우드 생성 함수(전처리 기반해서 작동)
def generate_wordcloud(text, title):
    wordcloud = WordCloud(
        width=800,
        height=400,
        max_words=200,
        background_color='white',
        font_path='/System/Library/Fonts/Supplemental/AppleSDGothicNeo.ttc'  # 한글 폰트 경로 설정
    ).generate(text)

    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()
    
    
    
# 한글 전처리 함수 예제 적용
positive_text = ' '.join(positive_reviews['content'].dropna().apply(preprocess_text_korean))
generate_wordcloud(positive_text, 'Positive Reviews (Korean)')

neutral_text = ' '.join(neutral_reviews['content'].dropna().apply(preprocess_text_korean))
generate_wordcloud(neutral_text, 'Neutral Reviews (Korean)')

negative_text = ' '.join(negative_reviews['content'].dropna().apply(preprocess_text_korean))
generate_wordcloud(negative_text, 'Negative Reviews (Korean)')

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



CalledProcessError: Command '['/usr/libexec/java_home']' returned non-zero exit status 1.